In [11]:
import heapq
import time
import pandas as pd
import sys
from pathlib import Path

# Adds the project root to the Python path so we can import citymap_data.py from src folder correctly
ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from src.citymap_data import (
    Phoenix_cities, Phoenix_heuristics,
    Los_Angeles_cities, Los_Angeles_heuristics,
    Las_Vegas_cities, Las_Vegas_heuristics,
    San_Diego_cities, San_Diego_heuristics
)

In [12]:
def uniform_cost_search(graph, start, goal):
    """
    Uniform Cost Search (UCS) algorithm.
    This algorithm explores the graph choosing always the lowest-cost path first.

    Returns:
        path (list): shortest path found from start node to goal node
        cost (float): the total cost of the path
        expanded (int): the number of the nodes explored/expanded
    """
    pq = [(0, start)] # Priority Queue ordered by the accumulation of the path cost
    costs = {start: 0} # The lowest cost found to reach each of the nodes
    parent = {start: None} # Parent node of each node to reconstruct the final path
    expanded = 0

    while pq:
        current_cost, current_node = heapq.heappop(pq)

        if current_node == goal:
            path = []
            while current_node is not None:
                path.append(current_node)
                current_node = parent[current_node]
            path.reverse()
            return path, current_cost, expanded

        expanded += 1

        # Explores all neighbours of the current node
        for neighbor, edge_cost in graph[current_node].items():
            new_cost = current_cost + edge_cost
            if neighbor not in costs or new_cost < costs[neighbor]:
                costs[neighbor] = new_cost
                parent[neighbor] = current_node
                heapq.heappush(pq, (new_cost, neighbor))

    return None, float("inf"), expanded


In [13]:

def a_star_search(graph, start, goal, heuristics):
    """
    A* search algorithm.
    This algorithm explores the graph using the actual path cost (g) and 
    the heuristic estimation (h) so the search becomes more efficient than UCS.

    Here, the priority queue is ordered by f = g + h, where:    
        - f(n) = estimated total cost of a path that goes through node n
        - g(n) = actual cost from the start node to node n
        - h(n) = heuristic estimate of the cost from node n to the goal

    Returns:
        path (list): shortest path found from start node to goal node
        cost (float): the total cost of the path
        expanded (int): the number of the nodes explored/expanded
    """   
    open_set = [(heuristics.get(start, 0), start)] 
    g_score = {start: 0}
    came_from = {start: None}
    expanded = 0

    while open_set:
        f_current, current = heapq.heappop(open_set)

        # If there is a better path for the current node, skip
        if f_current > g_score[current] + heuristics.get(current, 0):
            continue

        if current == goal:
            # Reconstruct path
            path = []
            node = current
            while node is not None:
                path.append(node)
                node = came_from[node]
            path.reverse()
            return path, g_score[current], expanded

        expanded += 1

        for neighbor, cost in graph[current].items():
            tentative_g = g_score[current] + cost

            if neighbor not in g_score or tentative_g < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g
                f_neighbor = tentative_g + heuristics.get(neighbor, 0)
                heapq.heappush(open_set, (f_neighbor, neighbor))

    return None, float("inf"), expanded


In [14]:
def round_trip(graph, office, news_location, heuristics, algorithm_out="ucs", algorithm_back="astar"):
    """
    This method computes a round trip for the given office to a news_location and back.


    Measures:
        - total cost of the trip
        - total execution time from office to news_location and back
        - number of nodes expanded
    """
    # Outbound path
    t0 = time.perf_counter()
    if algorithm_out == "ucs":
        path1, cost1, exp1 = uniform_cost_search(graph, office, news_location)
    else:
        path1, cost1, exp1 = a_star_search(graph, office, news_location, heuristics)
    t1 = time.perf_counter()

    # Return path
    t2 = time.perf_counter()
    if algorithm_back == "ucs":
        path2, cost2, exp2 = uniform_cost_search(graph, news_location, office)
    else:
        path2, cost2, exp2 = a_star_search(graph, news_location, office, heuristics)
    t3 = time.perf_counter()

    if path1 is None or path2 is None:
        return None

    return {
        "out_path": path1,
        "out_cost": cost1,
        "out_time_ms": (t1 - t0) * 1000,
        "out_expanded": exp1,
        "back_path": path2,
        "back_cost": cost2,
        "back_time_ms": (t3 - t2) * 1000,
        "back_expanded": exp2,
        # Combination of outbound and return paths, avoiding duplication of end/start back nodes
        "round_trip_path": path1 + path2[1:],
        "round_trip_cost": cost1 + cost2,
        "round_trip_time_ms": ((t1 - t0) + (t3 - t2)) * 1000,
        "round_trip_expanded": exp1 + exp2,
    }

In [15]:
# Combination of simple and complex scenarios to get a better analysis of the performance
SCENARIOS = [
    # PHOENIX
    ("Phoenix (simple)", Phoenix_cities, Phoenix_heuristics, "Phoenix", "Avondale"),  # direct/very short
    ("Phoenix (harder)", Phoenix_cities, Phoenix_heuristics, "Phoenix", "Tempe"),    # complex

    # LOS ANGELES
    ("Los Angeles (simple)", Los_Angeles_cities, Los_Angeles_heuristics, "Los Angeles", "Pasadena"),   # direct/short
    ("Los Angeles (harder)", Los_Angeles_cities, Los_Angeles_heuristics, "Los Angeles", "West Covina"),# complex

    # LAS VEGAS
    ("Las Vegas (simple)", Las_Vegas_cities, Las_Vegas_heuristics, "Las Vegas", "Boulder City"),  # short / near-direct
    ("Las Vegas (harder)", Las_Vegas_cities, Las_Vegas_heuristics, "Las Vegas", "Winchester"),    # complex

    # SAN DIEGO
    ("San Diego (simple)", San_Diego_cities, San_Diego_heuristics, "San Diego", "Chula Vista"),   # direct/short
    ("San Diego (harder)", San_Diego_cities, San_Diego_heuristics, "San Diego", "Carlsbad"),      # complex
]

# Prepares all combinations of algorithms UCS and A* to compare the heuristics impact
ALGORITHM_COMBOS = [
    ("ucs", "ucs"),
    ("ucs", "astar"),
    ("astar", "ucs"),
    ("astar", "astar"),
]

# Algorithm labels to improve readability of tables that will be displayed
ALGORITHM_LABEL = {
    "ucs": "UCS",
    "astar": "A*"
}

In [ ]:
rows = []

for city_name, graph, heur, office, news in SCENARIOS:
    for a_out, a_back in ALGORITHM_COMBOS:
        res = round_trip(graph, office, news, heur, algorithm_out=a_out, algorithm_back=a_back)

        # If any of the paths were not successful, skips this algorithms combination, or record it as failed
        if res is None:
            rows.append({
                "City": city_name,
                "Office": office,
                "News": news,
                "Algorithm": f"{ALGORITHM_LABEL[a_out]} (out) + {ALGORITHM_LABEL[a_back]} (back)",
                "RoundTripCost": None,
                "RoundTripTimeMs": None,
                "Expanded": None,
                # Path is stored for debugging, only displayed if "df_results" is displayed
                "Path": None
            })
            continue

        rows.append({
            "City": city_name,
            "Office": office,
            "News": news,
            "Algorithm": f"{ALGORITHM_LABEL[a_out]} (out) + {ALGORITHM_LABEL[a_back]} (back)",
            "RoundTripCost": res["round_trip_cost"],
            "RoundTripTimeMs": res["round_trip_time_ms"],
            "Expanded": res["round_trip_expanded"],
            # Path is stored for debugging, only displayed if "df_results" is displayed
            "Path": " -> ".join(res["round_trip_path"])
        })


df_results = pd.DataFrame(rows)
# Full table with excess of information, not displayed by default for convenience
# df_results

,City,Office,News,Algorithm,RoundTripCost,RoundTripTimeMs,Expanded,Path
0,Phoenix (simple),Phoenix,Avondale,UCS (out) + UCS (back),30,0.0208,6,Phoenix -> Avondale -> Phoenix
1,Phoenix (simple),Phoenix,Avondale,UCS (out) + A* (back),30,0.0096,5,Phoenix -> Avondale -> Phoenix
2,Phoenix (simple),Phoenix,Avondale,A* (out) + UCS (back),30,0.0111,6,Phoenix -> Avondale -> Phoenix
3,Phoenix (simple),Phoenix,Avondale,A* (out) + A* (back),30,0.0070,5,Phoenix -> Avondale -> Phoenix
4,Phoenix (harder),Phoenix,Tempe,UCS (out) + UCS (back),18,0.0056,3,Phoenix -> Tempe -> Phoenix
5,Phoenix (harder),Phoenix,Tempe,UCS (out) + A* (back),18,0.0042,2,Phoenix -> Tempe -> Phoenix
6,Phoenix (harder),Phoenix,Tempe,A* (out) + UCS (back),18,0.0045,3,Phoenix -> Tempe -> Phoenix
7,Phoenix (harder),Phoenix,Tempe,A* (out) + A* (back),18,0.0045,2,Phoenix -> Tempe -> Phoenix
8,Los Angeles (simple),Los Angeles,Pasadena,UCS (out) + UCS (back),22,0.0033,2,Los Angeles -> Pasadena -> Los Angeles
9,Los Angeles (simple),Los Angeles,Pasadena,UCS (out) + A* (back),22,0.0033,2,Los Angeles -> Pasadena -> Los Angeles


In [17]:
# Table to show the number of nodes expanded on each of the combinations
df_expanded = df_results.pivot_table(
    index=["City", "Office", "News"],
    columns="Algorithm",
    values="Expanded",
    aggfunc="first"
)
df_expanded

,,Algorithm,A* (out) + A* (back),A* (out) + UCS (back),UCS (out) + A* (back),UCS (out) + UCS (back)
City,Office,News,,,,
Las Vegas (harder),Las Vegas,Winchester,3,4,3,4
Las Vegas (simple),Las Vegas,Boulder City,13,16,14,17
Los Angeles (harder),Los Angeles,West Covina,9,9,9,9
Los Angeles (simple),Los Angeles,Pasadena,2,2,2,2
Phoenix (harder),Phoenix,Tempe,2,3,2,3
Phoenix (simple),Phoenix,Avondale,5,6,5,6
San Diego (harder),San Diego,Carlsbad,5,5,5,5
San Diego (simple),San Diego,Chula Vista,2,2,2,2


In [18]:
# Table to show the trip cost for all combinations
df_cost = df_results.pivot_table(
    index=["City", "Office", "News"],
    columns="Algorithm",
    values="RoundTripCost",
    aggfunc="first"
)
df_cost

,,Algorithm,A* (out) + A* (back),A* (out) + UCS (back),UCS (out) + A* (back),UCS (out) + UCS (back)
City,Office,News,,,,
Las Vegas (harder),Las Vegas,Winchester,12,12,12,12
Las Vegas (simple),Las Vegas,Boulder City,64,64,64,64
Los Angeles (harder),Los Angeles,West Covina,58,58,58,58
Los Angeles (simple),Los Angeles,Pasadena,22,22,22,22
Phoenix (harder),Phoenix,Tempe,18,18,18,18
Phoenix (simple),Phoenix,Avondale,30,30,30,30
San Diego (harder),San Diego,Carlsbad,50,50,50,50
San Diego (simple),San Diego,Chula Vista,20,20,20,20


In [19]:
# Table to show the execution time of the trips in millisecond for all combinations
df_time = df_results.pivot_table(
    index=["City", "Office", "News"],
    columns="Algorithm",
    values="RoundTripTimeMs",
    aggfunc="first"
)
df_time

,,Algorithm,A* (out) + A* (back),A* (out) + UCS (back),UCS (out) + A* (back),UCS (out) + UCS (back)
City,Office,News,,,,
Las Vegas (harder),Las Vegas,Winchester,0.0044,0.0054,0.0043,0.0051
Las Vegas (simple),Las Vegas,Boulder City,0.0092,0.0115,0.0082,0.0121
Los Angeles (harder),Los Angeles,West Covina,0.0066,0.0079,0.0063,0.0069
Los Angeles (simple),Los Angeles,Pasadena,0.0029,0.0031,0.0033,0.0033
Phoenix (harder),Phoenix,Tempe,0.0045,0.0045,0.0042,0.0056
Phoenix (simple),Phoenix,Avondale,0.0070,0.0111,0.0096,0.0208
San Diego (harder),San Diego,Carlsbad,0.0032,0.0031,0.0032,0.0030
San Diego (simple),San Diego,Chula Vista,0.0023,0.0022,0.0023,0.0021


In [20]:
BASE = ("ucs", "ucs") # Base algorithm used as reference to check if everything is right

# This loops compares if the paths returned by different combinations are the same as using UCS on both ways
for city_name, graph, heur, office, news in SCENARIOS:
    baseline = round_trip(graph, office, news, heur, algorithm_out="ucs", algorithm_back="ucs")
    if baseline is None:
        print(f"{city_name}: baseline UCS + UCS failed (no path)")
        continue
    
    # Variables to store base results for comparison
    baseline_path = baseline["round_trip_path"]
    baseline_cost = baseline["round_trip_cost"]

    all_same = True # Will be False if any algorithm is different than the baseline
    details = []    # Stores details of differences if there is any

    for a_out, a_back in ALGORITHM_COMBOS:        
        if (a_out, a_back) == BASE:
            # Skips the UCS + UCS combination, as it is the base case
            continue

        res = round_trip(graph, office, news, heur, algorithm_out=a_out, algorithm_back=a_back)
        if res is None:
            all_same = False
            details.append(
                f"{city_name}: {ALGORITHM_LABEL[a_out]} + {ALGORITHM_LABEL[a_back]} failed (no path)"
            )
            continue

        # Check whether this algorithm produced the same path and cost as the baseline
        same_path = (baseline_path == res["round_trip_path"])
        same_cost = (baseline_cost == res["round_trip_cost"])

        if not (same_path and same_cost):
            all_same = False
            details.append(
                f"{ALGORITHM_LABEL[a_out]} + {ALGORITHM_LABEL[a_back]} "
                f"produced a different result"
            )

    # Prints a single summary per scenario
    if all_same:
        print(f"{city_name}: all algorithm combinations produced the same path and cost as UCS + UCS.")
    else:
        print(f"{city_name}: differences detected compared to UCS + UCS.")
        for d in details:
            print(f"  - {d}")

Phoenix (simple): all algorithm combinations produced the same path and cost as UCS + UCS.
Phoenix (harder): all algorithm combinations produced the same path and cost as UCS + UCS.
Los Angeles (simple): all algorithm combinations produced the same path and cost as UCS + UCS.
Los Angeles (harder): all algorithm combinations produced the same path and cost as UCS + UCS.
Las Vegas (simple): all algorithm combinations produced the same path and cost as UCS + UCS.
Las Vegas (harder): all algorithm combinations produced the same path and cost as UCS + UCS.
San Diego (simple): all algorithm combinations produced the same path and cost as UCS + UCS.
San Diego (harder): all algorithm combinations produced the same path and cost as UCS + UCS.
